In [6]:
import sys

import rdkit # required for cheminformatics functionality (small molecules)
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [7]:
fp_length = 2048 # the length of the bit fingerprint (vector) representing the molecule

Create two lists from the data file, one with the molecules and the other with the classes of the molecules.  The classes are integers [0..10] representing the 11 classes.  Keep in mind that this is a multi-class problem, i.e. each  molecule (metabolite) may have more than one class.

In [8]:
# load the molecules
with open("../data/kegg_classes.txt") as f:
    mols_str, classes = zip(*[ line.strip().split() for line in f])

In [9]:
# generate the descriptors for ML 
fp_descriptors = [ AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(m), 2, nBits=fp_length) for m in mols_str ]

RDKit WARNING: [21:57:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:57:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:57:58] WARNING: not removing hydrogen atom with dummy atom neighbors
RDKit WARNING: [21:57:58] WARNING: not removing hydrogen atom with dummy atom neighbors


In [10]:
# cannot use list comprehensions here as ConvertToNumpyArray has a void return
ecfp = []
for d in fp_descriptors:
    np_arr = np.zeros(0, dtype=np.int8)
    DataStructs.ConvertToNumpyArray(d, np_arr)
    ecfp.append(np_arr)

# but what does one of these look like?
ecfp[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
ecfp = np.array(ecfp)
ecfp.shape

(6669, 2048)

In [12]:
# Convert classes to multi-class vector

multi_class_output = []

for elem in classes:
    
    my_class = np.zeros(11)
    
    for num in elem.split(","):
        my_class[int(num)] = 1
        
    multi_class_output.append(my_class)

In [13]:
multi_class_output = np.array(multi_class_output)

In [14]:
multi_class_output.shape

(6669, 11)

In [18]:
# Train deep neural network to perform multiclassp prediction from 2048 finger print
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.preprocessing import MinMaxScaler

In [20]:
scaler = MinMaxScaler()

ecfp = scaler.fit_transform(ecfp)

X_train, X_test, y_train, y_test = train_test_split(ecfp, multi_class_output, test_size=0.2)




In [25]:
model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(11, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, batch_size=10, validation_split=0.2)

Epoch 1/150
427/427 [==============================] - 8s 16ms/step - loss: 0.2908 - accuracy: 0.5062 - val_loss: 0.1567 - val_accuracy: 0.7451
Epoch 2/150
427/427 [==============================] - 6s 15ms/step - loss: 0.1066 - accuracy: 0.8076 - val_loss: 0.1289 - val_accuracy: 0.7601
Epoch 3/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0574 - accuracy: 0.8350 - val_loss: 0.1151 - val_accuracy: 0.7170
Epoch 4/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0383 - accuracy: 0.8331 - val_loss: 0.1342 - val_accuracy: 0.7573
Epoch 5/150
427/427 [==============================] - 7s 16ms/step - loss: 0.0322 - accuracy: 0.8483 - val_loss: 0.1235 - val_accuracy: 0.7563
Epoch 6/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0186 - accuracy: 0.8323 - val_loss: 0.1306 - val_accuracy: 0.7470
Epoch 7/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0169 - accuracy: 0.8570 - val_loss: 0.1243 - val_accuracy:

427/427 [==============================] - 6s 14ms/step - loss: 0.0027 - accuracy: 0.8673 - val_loss: 0.1820 - val_accuracy: 0.7760
Epoch 58/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0037 - accuracy: 0.8564 - val_loss: 0.1782 - val_accuracy: 0.7723
Epoch 59/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0036 - accuracy: 0.8547 - val_loss: 0.1791 - val_accuracy: 0.7798
Epoch 60/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0027 - accuracy: 0.8669 - val_loss: 0.1787 - val_accuracy: 0.7694
Epoch 61/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0024 - accuracy: 0.8623 - val_loss: 0.1844 - val_accuracy: 0.7713
Epoch 62/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0028 - accuracy: 0.8633 - val_loss: 0.1946 - val_accuracy: 0.7610
Epoch 63/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0099 - accuracy: 0.8597 - val_loss: 0.2389 - val_accuracy: 0.707

427/427 [==============================] - 6s 14ms/step - loss: 0.0094 - accuracy: 0.8450 - val_loss: 0.2804 - val_accuracy: 0.7404
Epoch 114/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0046 - accuracy: 0.8584 - val_loss: 0.2386 - val_accuracy: 0.7235
Epoch 115/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0118 - accuracy: 0.8425 - val_loss: 0.2342 - val_accuracy: 0.7479
Epoch 116/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0085 - accuracy: 0.8416 - val_loss: 0.2120 - val_accuracy: 0.7404
Epoch 117/150
427/427 [==============================] - 7s 15ms/step - loss: 0.0020 - accuracy: 0.8376 - val_loss: 0.2520 - val_accuracy: 0.7404
Epoch 118/150
427/427 [==============================] - 6s 14ms/step - loss: 0.0025 - accuracy: 0.8386 - val_loss: 0.2316 - val_accuracy: 0.7395
Epoch 119/150
427/427 [==============================] - 6s 15ms/step - loss: 0.0027 - accuracy: 0.8362 - val_loss: 0.2334 - val_accuracy:

In [26]:
y_pred = model.predict(X_test)

In [27]:
y_pred

array([[0.00000000e+00, 1.20801170e-31, 0.00000000e+00, 0.00000000e+00,
        3.35003983e-24, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 3.22702991e-30, 0.00000000e+00],
       [1.01208340e-18, 2.46542342e-10, 1.28135543e-05, 4.66650218e-10,
        1.98889433e-10, 1.00000000e+00, 3.39104189e-09, 1.07067823e-02,
        4.34534388e-11, 8.60127169e-09, 1.76891047e-06],
       [4.13981973e-34, 7.36947541e-27, 7.02983166e-19, 0.00000000e+00,
        1.78851199e-20, 0.00000000e+00, 0.00000000e+00, 4.51104784e-25,
        1.00000000e+00, 4.08256975e-23, 6.06429791e-28],
       [1.01812185e-15, 1.01347909e-16, 4.40152428e-11, 2.19594998e-20,
        1.00000000e+00, 4.99441330e-12, 4.69660974e-22, 4.54954893e-11,
        7.84723764e-15, 1.00000000e+00, 1.00154195e-12],
       [2.45576854e-18, 1.17576567e-28, 5.15877008e-10, 2.32752440e-14,
        3.08757345e-20, 1.31926952e-27, 4.94085561e-29, 1.76259764e-13,
        4.67424172e-18, 1.00000000e+00, 5.87704918e-

In [29]:
y_pred[y_pred<0.5]  = 0
y_pred[y_pred>=0.5] = 1

In [30]:
for c in range(11):
    y_true_c = y_test[:,c]
    y_pred_c = y_pred[:,c]
    
    auc       = accuracy_score(y_true_c, y_pred_c)
    precision = precision_score(y_true_c, y_pred_c)
    recall    = recall_score(y_true_c, y_pred_c)
    
    print('Class '+str(c+1)+' statistics:')
    print('Accuracy %.4f, Precision %.4f, Recall %.4f\n' %(auc, precision, recall))

Class 1 statistics:
Accuracy 0.9678, Precision 0.8839, Recall 0.9516

Class 2 statistics:
Accuracy 0.9820, Precision 0.9118, Recall 0.9451

Class 3 statistics:
Accuracy 0.9790, Precision 0.9091, Recall 0.9677

Class 4 statistics:
Accuracy 0.9813, Precision 0.7763, Recall 0.8806

Class 5 statistics:
Accuracy 0.9468, Precision 0.8494, Recall 0.9170

Class 6 statistics:
Accuracy 0.9790, Precision 0.8731, Recall 0.9141

Class 7 statistics:
Accuracy 0.9858, Precision 0.8571, Recall 0.8696

Class 8 statistics:
Accuracy 0.9693, Precision 0.8889, Recall 0.8736

Class 9 statistics:
Accuracy 0.9745, Precision 0.9493, Recall 0.9493

Class 10 statistics:
Accuracy 0.9535, Precision 0.9197, Recall 0.9197

Class 11 statistics:
Accuracy 0.9543, Precision 0.8794, Recall 0.9018

